# FX Forward Curves Fundamentals

## Namespaces

You can use namespaces in your Python by importing the necessary modules.

In [1]:
import datetime

from lseg_analytics.common import Description, TenorType, Quote, QuoteDefinition

from lseg_analytics.market_data.fx_forward_curves import (
    DepositFxConstituent,
    DepositConstituentDefinition,
    CurrencyBasisSwapConstituent,
    CurrencyBasisSwapConstituentDefinition,
    FxForwardConstituent,
    FxSpotConstituentDefinition,
    FxForwardConstituentDefinition,
    FxSpotConstituent,
    IndirectSourcesDeposits,
    IndirectSourcesSwaps,
    FxForwardCurve,
    FxForwardCurveDefinition,
    delete,
    load,
    search,
)

You can import api modules using their original names:

In [2]:
from lseg_analytics.market_data import fx_forward_curves

fx_forward_curves.__name__

'lseg_analytics.market_data.fx_forward_curves'

Or you can import them using an alias to prevent name conflicts:

In [3]:
from lseg_analytics.market_data import fx_forward_curves as ffc

ffc.FxForwardCurve, ffc.load

(lseg_analytics.market_data.fx_forward_curves._fx_forward_curve.FxForwardCurve,
 <function lseg_analytics.market_data.fx_forward_curves._functions.load(*, resource_id: Optional[str] = None, name: Optional[str] = None, space: Optional[str] = None)>)

## Constructors

Each object might have the following parameters:

- A definition containing information that *defines* a resource, used to build the resource object itself.
- A description containing information that *describes* a resource, for example a summary and/or tags.

The definition is mandatory while description is optional.


### Definition

You can construct objects in more than one way. 

In this case, define the FxForwardCurve with a simple 'definition':

In [4]:
# Create a FxForwardCurve instance.
new_curve = FxForwardCurve(
    FxForwardCurveDefinition(
        cross_currency="USDJPY", reference_currency="JPY"
    )
)

new_curve

<FxForwardCurve space=None name='' unsaved>

### Description

You can add the description later:

In [5]:
new_curve.description.summary="New curve"

new_curve.description.summary

'New curve'

## Builders

Curves can have a large number of parameters to enable them to cover many use cases. This provides flexibility, but makes curve creation complex.

The platform provides dedicated endpoints to create curves for specific use cases, allowing creation using only the parameters necessary for that use.

These endpoints are exposed as functions.

### Create FxForwardCurve from scratch.
This example creates a FX forward curve without using one of the dedicated functions.

In [6]:
curve_from_scratch = FxForwardCurve(
    description=Description(summary="My FX Forward Curve", tags=["tag1", "tag2"]),
    definition=FxForwardCurveDefinition(
        cross_currency="EURCHF",
        reference_currency="TRY",
        constituents=[
            FxSpotConstituent(
                quote=Quote(definition=QuoteDefinition(instrument_code="TRY=")),
                definition=FxSpotConstituentDefinition(template="USDEUR"),
            ),
            FxForwardConstituent(
                definition=FxForwardConstituentDefinition(template="USDEUR"),
            ),
            CurrencyBasisSwapConstituent(
                definition=CurrencyBasisSwapConstituentDefinition(
                    template="USDEUR", tenor="1W"
                ),
            ),
            DepositFxConstituent(
                definition=DepositConstituentDefinition(
                    tenor="1D", template="USDEUR"
                )
            ),
        ],
    ),
)

curve_from_scratch

<FxForwardCurve space=None name='' unsaved>

### Create definition from FXForwards
This example creates a curve from FXForwards.

In [7]:
from_fx_forwards = fx_forward_curves.create_from_fx_forwards(
            cross_currency="EURGBP",
            reference_currency="USD",
            sources=IndirectSourcesSwaps(base_fx_forwards="RFB"),
            additional_tenor_types=[TenorType.LONG, TenorType.END_OF_MONTH],
        )

from_fx_forwards

<FxForwardCurve space=None name='' unsaved>

### Create definition from deposits
This example creates a curve from Deposits.

In [8]:
from_deposits = fx_forward_curves.create_from_deposits(
            cross_currency="EURGBP",
            additional_tenor_types=[TenorType.LONG, TenorType.END_OF_MONTH],
        )

from_deposits

<FxForwardCurve space=None name='' unsaved>

## Save
This example illustrates how you can save a curve to a space.


In [9]:
from os import name


curve = FxForwardCurve(
    definition=from_fx_forwards.definition,
    description=Description(summary="EURGBP Fx Forward Curve via USD", tags=["EURGBP", "EUR", "GBP", "FxCurve"]),
)

curve

<FxForwardCurve space=None name='' unsaved>

In [10]:
curve.save(name="EURGBP_Fx_Forward_Curve", space="HOME")

True

## Clone

Clone the existing instance on definition and description.

In [11]:
cloned_fx_forward = curve.clone()
cloned_fx_forward.save(name="my_saved_curve2", space="HOME")

print(f"Curve_id: {curve.id}")
print(f"Cloned_fx_forwards_id: {cloned_fx_forward.id}")

delete(resource_id=cloned_fx_forward.id)

Curve_id: 6bd829d3-74b5-4bb8-bc71-249e6524f52a
Cloned_fx_forwards_id: 84c4b5ca-4ca9-46ce-81df-35039d4d10d3


True

## Location

Name and space are location attributes, which are automatically set when a resource object is saved for the first time.

Unsaved resources have thier name and space set to None.

Location attributes are read-only.

In [12]:
print(f"Curve name: {curve.location.name}")
print(f"Curve space: {curve.location.space}")

Curve name: EURGBP_Fx_Forward_Curve
Curve space: HOME


## Resource ID

A resource ID is the unique resource identifier for an object on the platform.

The resource ID is created on saving.

IDs are read-only.

In [13]:
curve_id = curve.id

curve_id

'6bd829d3-74b5-4bb8-bc71-249e6524f52a'

## Load

Previously saved resources can be loaded in either by name or by ID.

### By name

In [14]:
loaded_by_name_curve = load(name="EURGBP_Fx_Forward_Curve")
loaded_by_name_curve

<FxForwardCurve space='HOME' name='EURGBP_Fx_Forward_Curve' 6bd829d3‥>

### By ID

In [15]:
loaded_by_id_curve = load(resource_id=curve_id)
loaded_by_id_curve

<FxForwardCurve space='HOME' name='EURGBP_Fx_Forward_Curve' 6bd829d3‥>

Space is optional, but crucial if you need to have resources with the same name in different spaces:

In [16]:
loaded_by_name_and_space_curve = load(name="EURGBP_Fx_Forward_Curve", space="HOME")
loaded_by_name_and_space_curve

<FxForwardCurve space='HOME' name='EURGBP_Fx_Forward_Curve' 6bd829d3‥>

## Search

You can search for previously saved curves using the following techniques.

### Search for all curves

In [17]:
curves = search()

curves

[{'type': 'FxForwardCurve', 'id': '3b4e3e18-810b-43b1-8e87-de25cb1c658e', 'location': {'space': 'HOME', 'name': 'EURGBP_curve_from_FxForward_testnNVlpi'}, 'description': {'summary': 'My EURGBP curve from FxForward', 'tags': []}},
 {'type': 'FxForwardCurve', 'id': '6bd829d3-74b5-4bb8-bc71-249e6524f52a', 'location': {'space': 'HOME', 'name': 'EURGBP_Fx_Forward_Curve'}, 'description': {'summary': 'EURGBP Fx Forward Curve via USD', 'tags': ['EURGBP', 'EUR', 'GBP', 'FxCurve']}},
 {'type': 'FxForwardCurve', 'id': '07fbe09d-9968-4056-a87f-80b13c395547', 'location': {'space': 'HOME', 'name': 'GBPAUD_curve_from_FxForward_testIQs2kK'}, 'description': {'summary': 'My GBPAUD curve from FxForward', 'tags': []}},
 {'type': 'FxForwardCurve', 'id': '4bb6f784-18e1-4e98-a87f-14c5ea65aa2d', 'location': {'space': 'HOME', 'name': 'GBPAUD_curve_from_FxForward_testv2Padw'}, 'description': {'summary': 'My GBPAUD curve from FxForward', 'tags': []}},
 {'type': 'FxForwardCurve', 'id': '76b14964-65b3-49c9-97a0-ea

### Search for curves by names

In [18]:
curves_by_name = search(names=["EURGBP_Fx_Forward_Curve"])

new_curve

<FxForwardCurve space=None name='' unsaved>

### Search for curves by spaces

In [19]:
curves_by_spaces = search(spaces=["LSEG"])

curves_by_spaces

[{'type': 'FxForwardCurve', 'id': '8624ce0d-ad56-42ac-acd0-f7d21f32be2b', 'location': {'space': 'LSEG', 'name': 'AUD_AONIAUSD_SOFR_FxCross'}, 'description': {'summary': 'LSEG AUD AONIA/USD SOFR FxCross', 'tags': []}},
 {'type': 'FxForwardCurve', 'id': '3a45f4cb-29b7-4607-9f2e-7ab8a8086535', 'location': {'space': 'LSEG', 'name': 'AUD_BBSWUSD_SOFR_FxCross'}, 'description': {'summary': 'LSEG AUD BBSW/USD SOFR FxCross', 'tags': []}},
 {'type': 'FxForwardCurve', 'id': '1d3c2ac9-2761-499a-8df4-cc7e054bb029', 'location': {'space': 'LSEG', 'name': 'AUD_CAD_FxForward'}, 'description': {'summary': 'LSEG AUD CAD FxForward', 'tags': []}},
 {'type': 'FxForwardCurve', 'id': '9bac7c80-4d50-45ad-b66a-d935a8cf5655', 'location': {'space': 'LSEG', 'name': 'AUD_CHF_FxForward'}, 'description': {'summary': 'LSEG AUD CHF FxForward', 'tags': []}},
 {'type': 'FxForwardCurve', 'id': '11edca10-aa9b-4cc3-9b86-54d5d78119df', 'location': {'space': 'LSEG', 'name': 'AUD_JPY_FxForward'}, 'description': {'summary': 'LS

### Search for curves by tags

In [20]:
curves_by_tags = search(tags=["EUR", "USD"])

curves_by_tags

[]

### Combine search criteria
It's also possible to combine search items:

In [21]:
searched_curves = search(spaces=["LSEG"], tags=["EUR"])

searched_curves

[]

## Data access

You can access resource data through the root attributes of the resource object:

In [22]:
print(f"Curve cross currency: {curve.definition.cross_currency}")
print(f"Curve reference currency: {curve.definition.reference_currency}")


Curve cross currency: EURGBP
Curve reference currency: USD


## Curves functions

### Calculate

You can use Calculte to calculate an FxForward curve.

You can pass in the valuation date as a datetime.date object, or as a date string:

In [23]:
from lseg_analytics_basic_client.models import (
    FxForwardCurveCalculationParameters,
)

curve.calculate(
    pricing_preferences=FxForwardCurveCalculationParameters(
        valuation_date=datetime.date(2023, 10, 10)
    )
)

curve.calculate(
    pricing_preferences=FxForwardCurveCalculationParameters(valuation_date="2023-10-10")
)

{'resource': {'type': 'FxForwardCurve', 'id': '6bd829d3-74b5-4bb8-bc71-249e6524f52a', 'location': {'space': 'HOME', 'name': 'EURGBP_Fx_Forward_Curve'}, 'description': {'summary': 'EURGBP Fx Forward Curve via USD', 'tags': ['EURGBP', 'EUR', 'GBP', 'FxCurve']}, 'definition': {'crossCurrency': 'EURGBP', 'referenceCurrency': 'USD', 'constituents': [{'type': 'FxSpot', 'definition': {'template': 'EURUSD'}, 'quote': {'definition': {'instrumentCode': 'EUR=', 'source': 'Composite'}, 'values': {'bid': {'value': 1.1578}, 'ask': {'value': 1.1579}}}}, {'type': 'FxForward', 'definition': {'tenor': 'ON', 'template': 'EURUSD'}, 'quote': {'definition': {'instrumentCode': 'EURON=RFB', 'source': 'RFB'}, 'values': {'bid': {'value': 0.763}, 'ask': {'value': 0.765}}}}, {'type': 'FxForward', 'definition': {'tenor': 'TN', 'template': 'EURUSD'}, 'quote': {'definition': {'instrumentCode': 'EURTN=RFB', 'source': 'RFB'}, 'values': {'bid': {'value': 0.7617}, 'ask': {'value': 0.7637}}}}, {'type': 'FxForward', 'defi

## Delete

You can delete a curve from the server in the following ways:

### By ID:

In [24]:
delete(resource_id=curve_id)

True

### By name and space unique pair:

In [25]:
new_curve = FxForwardCurve(
    definition=FxForwardCurveDefinition(cross_currency="GPBEUR"),
    description=Description(summary="GPBCurve", tags=["GPB", "EUR"]),
)

new_curve.save(name="new_curve", space="newspace")

delete(name="new_curve", space="newspace")

True